<a href="https://colab.research.google.com/github/Anish32/color-detection-/blob/main/color_face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00


In [2]:

!pip install streamlit

In [3]:

!pip install streamlit

!pip install --upgrade gradio

!pip install opencv-python

!pip install --upgrade --force-reinstall gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0
  Using cached gradio-5.32.1-py3-none-any.whl.metadata (16 kB)
  Using cached gradio_client-1.10.2-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57

In [1]:
from IPython import get_ipython
from IPython.display import display
import gradio as gr
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
eye_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_eye.xml"
)

def detect_faces_eyes(image):
    """
    image: a NumPy array in RGB format (from Gradio)
    returns: annotated RGB image
    """

    frame = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(
            frame, "Face", (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2
        )

        roi_gray  = gray[y : y + h, x : x + w]
        roi_color = frame[y : y + h, x : x + w]
        eyes = eye_classifier.detectMultiScale(roi_gray, 1.1, 3)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(
                roi_color,
                (ex, ey),
                (ex + ew, ey + eh),
                (0, 255, 0),
                2
            )
            cv2.putText(
                roi_color, "Eye", (ex, ey - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1
            )

    return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

def detect_color_except_white(input_img):

    input_img_bgr = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)
    hsv_frame = cv2.cvtColor(input_img_bgr, cv2.COLOR_BGR2HSV)

    low = np.array([0, 42, 0])
    high = np.array([179, 255, 255])
    mask = cv2.inRange(hsv_frame, low, high)

    result_bgr = cv2.bitwise_and(input_img_bgr, input_img_bgr, mask=mask)

    input_img_rgb = cv2.cvtColor(input_img_bgr, cv2.COLOR_BGR2RGB)
    result_rgb = cv2.cvtColor(result_bgr, cv2.COLOR_BGR2RGB)


    return input_img_rgb, result_rgb
face_eye_interface = gr.Interface(
    fn=detect_faces_eyes,
    inputs=gr.Image(type="numpy", label="Upload Image"),
    outputs=gr.Image(label="Detected Faces & Eyes"),
    title="Face & Eye Detection"
)

color_detection_interface = gr.Interface(
    fn=detect_color_except_white,
    inputs=gr.Image(type="numpy", label="Upload or Capture Image"),
    outputs=[
        gr.Image(label="Original Image"),
        gr.Image(label="Detected Colors (excluding White)")
    ],
    title="Color Detection (Excluding White)",
    description="Upload or capture an image. This app will highlight all colors except white."
)
tabbed_app = gr.TabbedInterface(
    [face_eye_interface, color_detection_interface],
    ["Face and Eye Detection", "Color Detection"]
)

if __name__ == "__main__":
    tabbed_app.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3a4ddcbd5c636fc295.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [19]:
import zipfile
import os
import random
import shutil
from IPython import get_ipython
from IPython.display import display
import gradio as gr
import cv2
import numpy as np

zip_file_path = '/content/archive (2).zip'
extracted_dir = '/content/extracted_images'

os.makedirs(extracted_dir, exist_ok=True)

face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
eye_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_eye.xml"
)

def get_image_paths_from_zip(zip_path, extract_to):
    image_files = []
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Successfully extracted images to {extract_to}")

        for root, _, files in os.walk(extract_to):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                    image_files.append(os.path.join(root, file))

    except FileNotFoundError:
        print(f"Error: Zip file not found at {zip_path}")
    except zipfile.BadZipFile:
        print(f"Error: Bad zip file at {zip_path}")

    if not image_files:
        print("No image files found in the zip archive.")

    return image_files
all_image_paths = get_image_paths_from_zip(zip_file_path, extracted_dir)

if not all_image_paths:
    print("Exiting because no images were found to process.")
    exit()
def resize_image(image, max_size=(800, 600)):
    """
    Resizes an image while maintaining aspect ratio,
    limiting the maximum width and height.
    image: a NumPy array in RGB format
    returns: resized RGB image
    """
    if image is None:
        return None

    h, w = image.shape[:2]
    max_w, max_h = max_size

    if h > max_h or w > max_w:

        ratio = min(max_w / w, max_h / h)
        new_w = int(w * ratio)
        new_h = int(h * ratio)
        resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
        return resized_image
    else:
        return image
def detect_faces_eyes(image):
    """
    Performs face and eye detection on an image.
    image: a NumPy array in RGB format
    returns: annotated RGB image
    """
    if image is None:
        return None

    frame = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(
            frame, "Face", (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2
        )

        roi_gray  = gray[y : y + h, x : x + w]
        roi_color = frame[y : y + h, x : x + w]
        eyes = eye_classifier.detectMultiScale(roi_gray, 1.1, 3)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(
                roi_color,
                (ex, ey),
                (ex + ew, ey + eh),
                (0, 255, 0),
                2
            )
            cv2.putText(
                roi_color, "Eye", (ex, ey - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1
            )

    return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

def detect_color_except_white(input_img):
    """
    Detects colors excluding white in an image.
    input_img: a NumPy array in RGB format
    returns: result RGB image with colors excluding white highlighted
    """
    if input_img is None:
        return None

    input_img_bgr = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)
    hsv_frame = cv2.cvtColor(input_img_bgr, cv2.COLOR_BGR2HSV)

    low = np.array([0, 42, 0])
    high = np.array([179, 255, 255])
    mask = cv2.inRange(hsv_frame, low, high)

    result_bgr = cv2.bitwise_and(input_img_bgr, input_img_bgr, mask=mask)

    result_rgb = cv2.cvtColor(result_bgr, cv2.COLOR_BGR2RGB)

    return result_rgb

def process_random_image_with_detections():
    """
    Selects a random image, resizes it, and performs face/eye and color detection.
    Returns the original resized image, face/eye detected image, and color detected image.
    """
    if not all_image_paths:
        return None, None, None

    random_image_path = random.choice(all_image_paths)
    print(f"Randomly selected image: {random_image_path}")

    try:
        img_bgr = cv2.imread(random_image_path)
        if img_bgr is not None:
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

            resized_img_rgb = resize_image(img_rgb)

            if resized_img_rgb is None:
                 return None, None, None
            face_eye_detected_img = detect_faces_eyes(resized_img_rgb)

            color_detected_img = detect_color_except_white(resized_img_rgb)

            return resized_img_rgb, face_eye_detected_img, color_detected_img
        else:
            print(f"Could not load image from {random_image_path}")
            return None, None, None
    except Exception as e:
        print(f"An error occurred during image processing: {e}")
        return None, None, None

image_processing_interface = gr.Interface(
    fn=process_random_image_with_detections,
    inputs=None,
    outputs=[
        gr.Image(label="Original Resized Image", type="numpy"),
        gr.Image(label="Face & Eye Detection", type="numpy"),
        gr.Image(label="Color Detection (Excluding White)", type="numpy")
    ],
    title="Random Image Processing: Resize, Face/Eye, and Color Detection",
    description="Click the button to process a random image from the uploaded zip file with resizing, face/eye detection, and color detection."
)

if __name__ == "__main__":
    image_processing_interface.launch(inline=True, share=True)

Successfully extracted images to /content/extracted_images
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ac2306ac61b719c09a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [17]:
import zipfile
import os
import random
import shutil
from IPython import get_ipython
import gradio as gr
import cv2
import numpy as np

# Ensure Gradio is installed
try:
    import gradio as gr
except ImportError:
    print("Gradio not found. Installing Gradio...")
    get_ipython().run_line_magic('pip', 'install gradio')
    import gradio as gr

# Ensure OpenCV is installed
try:
    import cv2
except ImportError:
    print("OpenCV not found. Installing opencv-python...")
    get_ipython().run_line_magic('pip', 'install opencv-python')
    import cv2

# Define the path to your zip file (for the random image processing part)
zip_file_path = '/content/archive (2).zip' # Make sure this path is correct

# Define the directory to extract the images
extracted_dir = '/content/extracted_images'

# Create the extraction directory if it doesn't exist
os.makedirs(extracted_dir, exist_ok=True)

# Load cascade classifiers for face and eye detection
# Double-check cascade file paths
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
eye_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_eye.xml" # Corrected path
)


# Function to extract images and get a list of image paths (for random processing)
def get_image_paths_from_zip(zip_path, extract_to):
    image_files = []
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Successfully extracted images to {extract_to}")

        for root, _, files in os.walk(extract_to):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                    image_files.append(os.path.join(root, file))

    except FileNotFoundError:
        print(f"Error: Zip file not found at {zip_path}")
    except zipfile.BadZipFile:
        print(f"Error: Bad zip file at {zip_path}")

    if not image_files:
        print("No image files found in the zip archive.")

    return image_files

# Get the list of image paths from the zip file (for random processing)
# This needs to be done before the Gradio app is launched
all_image_paths = get_image_paths_from_zip(zip_file_path, extracted_dir)

# --- Image Processing Functions (used by both random and upload methods) ---

def resize_image(image, max_size=(800, 600)):
    """
    Resizes an image while maintaining aspect ratio,
    limiting the maximum width and height.
    image: a NumPy array in RGB format
    returns: resized RGB image
    """
    if image is None:
        return None

    h, w = image.shape[:2]
    max_w, max_h = max_size

    if h > max_h or w > max_w:
        # Calculate the ratio
        ratio = min(max_w / w, max_h / h)
        new_w = int(w * ratio)
        new_h = int(h * ratio)
        resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
        return resized_image
    else:
        return image # No resizing needed

def detect_faces_eyes(image):
    """
    Performs face and eye detection on an image.
    image: a NumPy array in RGB format
    returns: annotated RGB image
    """
    if image is None:
        return None

    frame = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Ensure classifiers are loaded
    if face_classifier.empty() or eye_classifier.empty():
        print("Error: Cascade classifiers not loaded.")
        return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Return original frame if classifiers fail

    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(
            frame, "Face", (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2
        )

        roi_gray  = gray[y : y + h, x : x + w]
        roi_color = frame[y : y + h, x : x + w]
        eyes = eye_classifier.detectMultiScale(roi_gray, 1.1, 3)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(
                roi_color,
                (ex, ey),
                (ex + ew, ey + eh),
                (0, 255, 0),
                2
            )
            cv2.putText(
                roi_color, "Eye", (ex, ey - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1
            )

    return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

def detect_color_except_white(input_img):
    """
    Detects colors excluding white in an image.
    input_img: a NumPy array in RGB format
    returns: result RGB image with colors excluding white highlighted
    """
    if input_img is None:
        return None

    # Gradio provides the input image in RGB, convert to BGR for OpenCV processing
    input_img_bgr = cv2.cvtColor(input_img, cv2.COLOR_RGB2BGR)
    hsv_frame = cv2.cvtColor(input_img_bgr, cv2.COLOR_BGR2HSV)

    low = np.array([0, 42, 0])
    high = np.array([179, 255, 255])
    mask = cv2.inRange(hsv_frame, low, high)

    result_bgr = cv2.bitwise_and(input_img_bgr, input_img_bgr, mask=mask)

    # Convert results back to RGB for Gradio
    result_rgb = cv2.cvtColor(result_bgr, cv2.COLOR_BGR2RGB)

    return result_rgb

# --- Gradio Functions for Interface ---

def process_uploaded_image_video(input_media):
    """
    Processes uploaded image or video for face and eye detection.
    input_media: Image (NumPy array) or Video (file path string) from Gradio
    returns: annotated media (image or video)
    """
    if input_media is None:
        return None

    # If it's an image (NumPy array)
    if isinstance(input_media, np.ndarray):
        # Perform face and eye detection on the image
        return detect_faces_eyes(input_media)

    # If it's a video (file path string)
    elif isinstance(input_media, str):
        # Use a generator function for video processing in Gradio
        def video_frame_generator(video_path):
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"Error opening video file: {video_path}")
                return # Use return instead of yield when opening fails

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                # Convert frame to RGB for processing and Gradio
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                annotated_frame = detect_faces_eyes(frame_rgb)
                yield annotated_frame

            cap.release()
            print("Video processing finished.")

        return video_frame_generator(input_media)


def process_uploaded_image_color(input_image):
    """
    Processes uploaded image for color detection (excluding white).
    input_image: Image (NumPy array) from Gradio
    returns: result RGB image
    """
    if input_image is None:
        return None

    return detect_color_except_white(input_image)


def process_random_image_with_detections():
    """
    Selects a random image, resizes it, and performs face/eye and color detection.
    Returns the original resized image, face/eye detected image, and color detected image.
    """
    if not all_image_paths:
        return None, None, None

    # Select a random image file
    random_image_path = random.choice(all_image_paths)
    print(f"Randomly selected image: {random_image_path}")

    try:
        img_bgr = cv2.imread(random_image_path)
        if img_bgr is not None:
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

            # Resize the image
            resized_img_rgb = resize_image(img_rgb)

            if resized_img_rgb is None:
                 return None, None, None # Handle potential resizing error

            # Perform face and eye detection on the resized image
            face_eye_detected_img = detect_faces_eyes(resized_img_rgb)

            color_detected_img = detect_color_except_white(resized_img_rgb)

            return resized_img_rgb, face_eye_detected_img, color_detected_img
        else:
            print(f"Could not load image from {random_image_path}")
            return None, None, None
    except Exception as e:
        print(f"An error occurred during image processing: {e}")
        return None, None, None

with gr.Blocks() as upload_face_eye_interface:
    gr.Markdown("### Upload: Face & Eye Detection (Image/Video)")
    gr.Markdown("Upload an image or record a short video to detect faces and eyes.")
    upload_input = gr.Image(type="numpy", label="Upload Image or Video", streaming=True)
    upload_output = gr.Image(label="Detected Faces & Eyes")
    upload_button = gr.Button("Run Detection")
    upload_button.click(fn=process_uploaded_image_video, inputs=upload_input, outputs=upload_output)

with gr.Blocks() as upload_color_interface:
    gr.Markdown("### Upload: Color Detection (Excluding White) - Image Only")
    gr.Markdown("Upload or capture an image for color detection.")
    color_input = gr.Image(type="numpy", label="Upload or Capture Image")
    color_output = gr.Image(label="Detected Colors (excluding White)")
    color_button = gr.Button("Run Detection")
    color_button.click(fn=process_uploaded_image_color, inputs=color_input, outputs=color_output)

with gr.Blocks() as random_image_interface:
    gr.Markdown("### Random from Zip: Resize, Face/Eye, and Color Detection")
    gr.Markdown("Click the button to process a random image from the uploaded zip file with resizing, face/eye detection, and color detection.")
    with gr.Row():
        random_button = gr.Button("Process Random Image")
    with gr.Row():
        original_image_output = gr.Image(label="Original Resized Image", type="numpy")
        face_eye_output = gr.Image(label="Face & Eye Detection", type="numpy")
        color_output = gr.Image(label="Color Detection (Excluding White)", type="numpy")

    random_button.click(
        fn=process_random_image_with_detections,
        inputs=None,
        outputs=[original_image_output, face_eye_output, color_output]
    )

tabbed_app = gr.TabbedInterface(
    [upload_face_eye_interface, upload_color_interface, random_image_interface],
    ["Upload: Face/Eye", "Upload: Color", "Random Images"]
)

if __name__ == "__main__":
    if os.path.exists(extracted_dir):
         shutil.rmtree(extracted_dir)
         print(f"Cleaned up previous extracted directory: {extracted_dir}")

    all_image_paths = get_image_paths_from_zip(zip_file_path, extracted_dir)

    if not all_image_paths:
        print("Warning: No images found in the zip file. The 'Random from Zip' tab will not function.")

    tabbed_app.launch(inline=True, share=True)


Successfully extracted images to /content/extracted_images
Cleaned up previous extracted directory: /content/extracted_images
Successfully extracted images to /content/extracted_images
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0471f34e78c42b2eaa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
